2. Pythonではじめるクローリング・スクレイピング

  - 2-1. Pythonを使うメリット
  - 2-2. Pythonのインストールと実行
  - 2-3. Pythonの基礎知識
  - 2-4. Webページを取得する
  - 2-5. Webページからデータを抜き出す
  - 2-6. データをファイルに保存する
  - 2-7. Pythonによるスクレイピングの流れ
  - 2-8. URLの基礎知識
  - 2-9. まとめ

## 2-4. Webページを取得する

### 2-4-1. RequestsによるWebページの取得

In [1]:
import requests

r = requests.get(("https://gihyo.jp/dp"))

In [2]:
# get()関数の戻り値はResponseオブジェクト
type(r)

requests.models.Response

In [4]:
# status_code属性でHTTPステータスコードを取得できる
r.status_code

200

In [5]:
# header属性でHTTPヘッダーの辞書を取得できる
r.headers["content-type"]

'text/html; charset=UTF-8'

In [6]:
# encoding属性でHTTPヘッダーから得られたエンコーディングを取得できる
r.encoding

'UTF-8'

In [7]:
# text属性でstr型にデコードしたレスポンスボディを取得できる
r.text

'<!DOCTYPE HTML>\n<html lang="ja" class="pc">\n<head>\n  <meta charset="UTF-8">\n  <title>Gihyo Digital Publishing … 技術評論社の電子書籍</title>\n  <meta http-equiv="Content-Style-Type" content="text/css"/>\n  <meta http-equiv="Content-Script-Type" content="application/javascript"/>\n  <meta name="description" content="技術評論社の電子書籍（電子出版）販売サイト"/>\n  <meta name="keywords" content="電子書籍,電子出版,EPUB,PDF,技術評論社"/>\n  <meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1"/>\n  <meta name="apple-mobile-web-app-capable" content="yes"/>\n  <meta name="format-detection" content="telephone=no"/>\n  <link rel="related" href="http://gihyo.jp/dp/catalogs.opds" type="application/atom+xml;profile=opds-catalog" title="Gihyo Digital Publishing OPDS Catalog"/>\n  <link rel="shortcut icon" href="/assets/templates/gdp/favicon.ico" type="image/vnd.microsoft.icon"/>\n  <link rel="apple-touch-icon-precomposed" href="/dp/assets/gdp-icon.png"/>\n  <!--[if lt IE 9]>\n    <script>var msie=8;</script>\n    <script src="/

#### Requestsの高度な機能

In [8]:
# GitHub REST API v3 で、RequestsのリポジトリをJSON形式で取得する
r = requests.get("https://api.github.com/repos/psf/requests")
r.json()

{'id': 1362490,
 'node_id': 'MDEwOlJlcG9zaXRvcnkxMzYyNDkw',
 'name': 'requests',
 'full_name': 'psf/requests',
 'private': False,
 'owner': {'login': 'psf',
  'id': 50630501,
  'node_id': 'MDEyOk9yZ2FuaXphdGlvbjUwNjMwNTAx',
  'avatar_url': 'https://avatars.githubusercontent.com/u/50630501?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/psf',
  'html_url': 'https://github.com/psf',
  'followers_url': 'https://api.github.com/users/psf/followers',
  'following_url': 'https://api.github.com/users/psf/following{/other_user}',
  'gists_url': 'https://api.github.com/users/psf/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/psf/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/psf/subscriptions',
  'organizations_url': 'https://api.github.com/users/psf/orgs',
  'repos_url': 'https://api.github.com/users/psf/repos',
  'events_url': 'https://api.github.com/users/psf/events{/privacy}',
  'received_events_url': 'https://api.github.com/

In [11]:
# httpbin.orgというHTTPリクエスト／レスポンスを試せるサービスを検証に用いる。
# POSTメソッドで送信。
# キーワード引数dataにdictを指定するとHTMLフォーム形式で送信される。
r = requests.post("http://httpbin.org/post", data={"key1": "value1"})
r

<Response [200]>

In [13]:
# リクエストに追加するHTTPヘッダーとキーワード引数headerにdictで指定する。
r = requests.get(
    "http://httpbin.org/get",
    headers={"user-agent": "my-crawler/1.0 (+foo@example.com)"}
)
r

<Response [200]>

In [ ]:
# Basic認証のユーザー名とパスワードの組をキーワード引数authで指定する。
r = requests.get(
    "https://api.github.com.user",
    auth=("<user-id>", "<user-pass>")
)

In [ ]:
# URLのパラメータはキーワード引数paramsで指定することも可能。
r = requests.get("http://httpbin.org/get", params={"key1": "value1"})

In [ ]:
# HTTPヘッダーを複数のリクエストで使い回す
s = requests.Session()
s.headers.update({"user-agent": "my-crawler/1.0 (+foo@example.com)"})

# Sessionオブジェクトにはget(), post()などのメソッドがあり、
# requests.get(), requests.pot()などと同様に使える。
r = s.get("https://gihyo.jp/")
r = s.get("https://gihyo.jp/dp")

### 2-4-2 文字コードの扱い

In [14]:
# UTF-8でエンコードした場合、E3 81 82 という3バイト
"あ".encode("UTF-8")

b'\xe3\x81\x82'

In [15]:
# CP932でエンコードした場合、82 A0 という2バイト
"あ".encode("cp932")

b'\x82\xa0'

In [16]:
# EUC-JPでエンコードした場合、A4 A2 という2バイト
"あ".encode("euc-jp")

b'\xa4\xa2'

#### Webページのエンコーディングを取得・推定する方法

1. HTTPレスポンスのContent-Typeヘッダーのcharsetで指定されたエンコーディングを取得する。
  - 正しくなかったり、charsetがしてされていないことがある。
2. HTTPレスポンスボディのバイト列の特徴からエンコーディングを推定する。
  - 推定に使用するレスポンスボディが長いほど、精度良く推定できる。（処理時間は長くなる）
3. HTMLのmetaタグで指定されたエンコーディングを取得する。
  - 1. が正しくしてされているサイトではmetaタグが指定されていないことがある。

#### HTTPヘッダーからエンコーディングを取得する

- 日本語ページの典型的なContent-Typeヘッダーの値
  - text/html
  - text/html; charset=UTF-8
  - text/html; charset=EUC-JP

#### レスポンスボディのバイト列からエンコーディングを推定する

#### meta タグからエンコーディングを取得する

### 2-5. Webページからデータを抜き出す

- 正規表現
  - HTMLを単純な文字列をみなして必要な部分を抜き出す
- HTMLパーサー
  - HTMLのタグを解析（パース）して必要な部分を抜き出す

#### 2-5-1 正規表現によるスクレイピング

- re モジュールの使い方
- https://docs.python.org/3/library/re.html